In [1]:
from __future__ import annotations

import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    TableMap,
    SubTableMap,
    SubMap,
    SelIndex,
    DataBase
)

In [3]:
from test_db_schema import (
    Search,
    Project,
    Task,
    User,
    Membership,
    Organization,
    SearchResult,
    TestSchema,
)

In [4]:
from typing import Any


async def dummy_loader(obj: dict[str, Any]) -> dict:
    await sleep(0.2)
    return {
        'name': obj["name"],
        'role': obj["role"],
        'age': obj["age"] + 1
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": SubTableMap(
            target=Search.results,
            rel_map=TableMap(push={SearchResult.score}),
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": SubTableMap(
                    target=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": SubTableMap(
                            target=Task.assignees,
                            push=User.name,
                            match_by=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": SubTableMap(
                    target=Project.members,
                    loader=dummy_loader,
                    push={User.name, User.age},
                    match_by=User.name,
                    rel_map=TableMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                    match_by=[Organization.name, Organization.city],
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
from datetime import date


data = {
    "resultCount": 3,
    "search": "test",
    "results": [
        {
            "score": 0.9,
            "project_name": "baking cake",
            "project_start": date(2020, 1, 1),
            "project_end": date(2020, 1, 4),
            "project_status": "done",
            "organization_name": "Bakery",
            "organization_address": "Main Street 1",
            "organization_city": "Bakerville",
            "tasks": [
                {
                    "task_name": "task1",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task2",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task3",
                    "task_assignees": ["Jane"],
                    "task_status": "done",
                },
            ],
            "members": [
                {"name": "John", "role": "baker", "age": 30},
                {"name": "John", "role": "manager", "age": 40},
            ],
        },
        {
            "score": 0.8,
            "project_name": "cleaning shoes",
            "project_start": date(2020, 1, 2),
            "project_end": date(2020, 1, 5),
            "project_status": "done",
            "organization_name": "Shoe Shop",
            "organization_address": "Main Street 2",
            "organization_city": "Shoetown",
            "tasks": [
                {
                    "task_name": "task4",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task5",
                    "task_assignees": ["Jane"],
                    "task_status": "todo",
                },
            ],
            "members": [
                {"name": "John", "role": "cleaner", "age": 25},
                {"name": "Jane", "role": "manager", "age": 35},
            ],
        },
        {
            "score": 0.7,
            "project_name": "fixing cars",
            "project_start": date(2020, 1, 3),
            "project_end": date(2020, 1, 6),
            "project_status": "started",
            "organization_name": "Car Shop",
            "organization_address": "Main Street 3",
            "organization_city": "Cartown",
            "tasks": [
                {
                    "task_name": "task6",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                }
            ],
            "members": [
                {"name": "John", "role": "mechanic", "age": 45},
                {"name": "Jane", "role": "manager", "age": 55},
                {"name": "Jack", "role": "manager", "age": 65},
            ],
        },
    ],
}

In [7]:
import warnings

with warnings.catch_warnings():
  warnings.simplefilter("error")
  with cProfile.Profile() as pr:
    db = DataBase(schema=TestSchema)
    rec = await data_source.load([data], db=db)

    stats = pstats.Stats(pr)

In [8]:
db[User].values()


[
    User({'name': 'John', 'age': 31, '_id': '7e62f88a-0d0b-4007-8afc-0185039115d0'}),
    User({'name': 'Jane', 'age': 56, '_id': '6a8ef8f2-12f5-400a-950c-2746d67885b7'}),
    User({'name': 'Jack', 'age': 66, '_id': '83dd954f-e41d-4de6-9ba0-28733381aa2a'})
]

In [9]:
db[Search.term].values()

['test']

In [10]:
list(db[Search.result_count].items())

[('test', 3)]

In [11]:
list(db[Search.results].items())


[
    (
        ('test', 1),
        Project({'number': 1, 'name': 'cleaning shoes', 'start': datetime.date(2020, 1, 2), 'end': datetime.date(2020, 1, 5), 'status': 'done'})
    ),
    (
        ('test', 2),
        Project({'number': 2, 'name': 'fixing cars', 'start': datetime.date(2020, 1, 3), 'end': datetime.date(2020, 1, 6), 'status': 'started'})
    ),
    (
        ('test', 0),
        Project({'number': 0, 'name': 'baking cake', 'start': datetime.date(2020, 1, 1), 'end': datetime.date(2020, 1, 4), 'status': 'done'})
    )
]

In [12]:
list(db[Search.results].items())[0][1].start.year

2020

In [13]:
db[Search.results].df()

term,test_db_schema_Project_number,test_db_schema_Organization__id,name,number,start,status,end
str,i64,str,str,i64,date,str,date
"""test""",1,"""06c09942-827d-4f13-9509-4f3b6e…","""cleaning shoes""",1,2020-01-02,"""done""",2020-01-05
"""test""",2,"""35eb9589-3d15-486c-8ff8-ab19d1…","""fixing cars""",2,2020-01-03,"""started""",2020-01-06
"""test""",0,"""e9638a6d-b02c-4463-a3d0-134d87…","""baking cake""",0,2020-01-01,"""done""",2020-01-04


In [14]:
db[Organization].df()

_id,name,address,city
str,str,str,str
"""e9638a6d-b02c-4463-a3d0-134d87…","""Bakery""","""Main Street 1""","""Bakerville"""
"""35eb9589-3d15-486c-8ff8-ab19d1…","""Shoe Shop""","""Main Street 2""","""Shoetown"""
"""06c09942-827d-4f13-9509-4f3b6e…","""Car Shop""","""Main Street 3""","""Cartown"""


In [16]:
db.describe()


{
    'schema': {
        'package': 'py-research',
        'module': 'test_db_schema',
        'object': 'TestSchema',
        'package_version': '^5.1.1.dev133+gf6dad64.d20250116',
        'repo': 'https://github.com/cloudlane-one/py-research.git',
        'repo_revision': '539a4a1f31004022ad53addbbe18e16c0f725a2c',
        'module_path': WindowsPath('exp/test_db_schema.py'),
        'module_dirty': True,
        'docs_url': 'https://cloudlane-one.github.io/py-research/'
    },
    'contents': {
        'records': {
            'test_db_schema.User': 3,
            'test_db_schema.Task': 6,
            'test_db_schema.Search': 1,
            'test_db_schema.Organization': 3,
            'test_db_schema.Project': 3
        },
        'arrays': {'test_db_schema_Organization_countries': 0},
        'relations': {'test_db_schema.Membership': 7, 'test_db_schema.SearchResult': 3}
    },
    'backend': None
}

In [17]:
db[Organization.countries].values()

c:\Users\lworm\Repos\py-research\.venv\Lib\site-packages\polars\io\database\_executor.py:501: SAWarning: SELECT statement has a cartesian product between FROM element(s) "test_db_schema_Organization" and FROM element "types_test_db_schema_Organization_countries".  Apply join condition(s) between each element to resolve.
  result = cursor_execute(query, **options)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ c:\Users\lworm\Repos\py-research\.venv\Lib\site-packages\sqlalchemy\engine\base.py:1964 in       │
│ _exec_single_context                                                                             │
│                                                                                                  │
│   1961 │   │   │   │   │   │   │   evt_handled = True                                            │
│   1962 │   │   │   │   │   │   │   break                                                         │
│   1963 │   │   │   │   if not evt_handled:                                                       │
│ ❱ 1964 │   │   │   │   │   self.dialect.do_execute(                                              │
│   1965 │   │   │   │   │   │   cursor, str_statement, effective_parameters, context              │
│   1966 │   │   │   │   │   )                                                                     │
│   1967                                                                                           │
│                                                                                                  │
│ c:\Users\lworm\Repos\py-research\.venv\Lib\site-packages\sqlalchemy\engine\default.py:942 in     │
│ do_execute                                                                                       │
│                                                                                                  │
│    939 │   │   cursor.executemany(statement, parameters)                                         │
│    940 │                                                                                         │
│    941 │   def do_execute(self, cursor, statement, parameters, context=None):                    │
│ ❱  942 │   │   cursor.execute(statement, parameters)                                             │
│    943 │                                                                                         │
│    944 │   def do_execute_no_params(self, cursor, statement, context=None):                      │
│    945 │   │   cursor.execute(statement)                                                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
OperationalError: no such column: test_db_schema_Organization.countries

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 db[Organization.countries].values()                                                          │
│   2                                                                                              │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:734 in values                   │
│                                                                                                  │
│    731 │   │   self: Data[ValT2, Any, Any, Any, Any, DynBackendID],                              │
│    732 │   ) -> Sequence[ValT2]:                                                                 │
│    733 │   │   """Iterable over this dataset's values."""                                        │
│ ❱  734 │   │   dfs = self.df()                                                                   │
│    735 │   │   if isinstance(dfs, pl.DataFrame):                                                 │
│    736 │   │   │   dfs = (dfs,)                                                                  │
│    737                                                                                           │
│                  

In [15]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         541574 function calls (507694 primitive calls) in 1.147 seconds



   Ordered by: cumulative time
   List reduced from 2270 to 143 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       52    0.001    0.000    0.527    0.010 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:690(_load_rec_from_item)
       52    0.002    0.000    0.523    0.010 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:569(_load_record)
       23    0.000    0.000    0.416    0.018 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1368(__ior__)
       23    0.001    0.000    0.415    0.018 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2794(_mutate)
       56    0.000    0.000    0.378    0.007 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:831(load)
   168/57    0.001    0.000    0.348    0.006 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:658(_load_records)
16924/8403    0.029    0.000    0.300    0.000 C:\User